In [1]:
api_url = "https://earth-search.aws.element84.com/v1"
from pystac_client import Client
client = Client.open(api_url)
collection = "sentinel-2-l2a"  # Sentinel-2, Level 2A, Cloud Optimized GeoTiffs (COGs)

In [8]:
from pygeotile.tile import Tile
from pygeotile.point import Point as pyPoint
from shapely.geometry import Point, Polygon

tms_x, tms_y, zoom = 16282, 11003, 15
tile = Tile.from_google(google_x=tms_x, google_y=tms_y, zoom=zoom)  # Tile Map Service (TMS) X Y and zoom
print(tile.bounds[0])
polygon = Polygon([
    Point(tile.bounds[0].longitude,tile.bounds[0].latitude),
    Point(tile.bounds[1].longitude,tile.bounds[0].latitude),
    Point(tile.bounds[1].longitude,tile.bounds[1].latitude),
    Point(tile.bounds[0].longitude,tile.bounds[1].latitude)
])


print(tile.bounds)
print('Bounds: ',polygon.wkt)

Point(latitude=50.76425935711649, longitude=-1.120605468749986)
(Point(latitude=50.76425935711649, longitude=-1.120605468749986), Point(latitude=50.77120783188783, longitude=-1.109619140624995))
Bounds:  POLYGON ((-1.120605468749986 50.76425935711649, -1.109619140624995 50.76425935711649, -1.109619140624995 50.77120783188783, -1.120605468749986 50.77120783188783, -1.120605468749986 50.76425935711649))


In [13]:
import numpy as np
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling

import rasterio.plot
import rasterio.mask
import os
import urllib
search = client.search(
    collections=[collection],
    intersects=polygon,
    max_items=100,
    sortby=[
            {"direction": "desc", "field": "properties.datetime"},
            {"direction": "asc", "field": "id"}
    ]
)

dst_crs = 'EPSG:3857'

print(search.matched())
item = [item for item in search.items() if item.properties['eo:cloud_cover'] < 20][0]
#print(item.__dict__)
print(item.properties['eo:cloud_cover'])
print(item.id)
file_loc = item.id+".tif"
file_dest = str(tms_x)+str(tms_y)+str(zoom)+".tif"
if (not os.path.isfile(file_loc)):
    urllib.request.urlretrieve(item.assets['visual'].href, file_loc)#Would be worth returning Scene Classification Layer (SCL)
with rasterio.open(file_loc) as dataset:
    #rasterio.plot.show(dataset)
    #reprojectn('EPSG:'+str(item.properties['proj:epsg']),'EPSG:3857')
    transform, width, height = calculate_default_transform(
    dataset.crs, dst_crs, dataset.width, dataset.height, *dataset.bounds)
    kwargs = dataset.meta.copy()
    kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })
    with rasterio.open(file_dest, 'w', **kwargs) as dst:
        for i in range(1, dataset.count + 1):
            reproject(
                source=rasterio.band(dataset, i),
                destination=rasterio.band(dst, i),
                src_transform=dataset.transform,
                src_crs=dataset.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.bilinear)
    for x in range(16282,16290):
        for y in range(11000,11010):
            tile = Tile.from_google(google_x=x, google_y=y, zoom=zoom)  # Tile Map Service (TMS) X Y and zoom
           
            with rasterio.open(file_dest) as src:
                polygon_3857 = Polygon([
                    pyPoint.from_latitude_longitude(latitude=tile.bounds[0].latitude,longitude=tile.bounds[0].longitude).meters,
                    pyPoint.from_latitude_longitude(latitude=tile.bounds[0].latitude,longitude=tile.bounds[1].longitude).meters,
                    pyPoint.from_latitude_longitude(latitude=tile.bounds[1].latitude,longitude=tile.bounds[1].longitude).meters,
                    pyPoint.from_latitude_longitude(latitude=tile.bounds[1].latitude,longitude=tile.bounds[0].longitude).meters,
                ])
                print(polygon_3857)
                print(src.bounds)
                print(src.crs)
                file_dest_cropped = str(x)+str(y)+str(zoom)+".tif"
                out_image, out_transform = rasterio.mask.mask(src, [polygon_3857], crop=True)
                out_meta = src.meta
                out_meta.update({"driver": "GTiff",
                         "height": out_image.shape[1],
                         "width": out_image.shape[2],
                         "transform": out_transform})
                with rasterio.open("cropped-"+file_dest_cropped, "w", **out_meta) as dest:
                    dest.write(out_image)

504
0.009194
S2B_30UXB_20230526_0_L2A
POLYGON ((-124745.23016140609 6583368.372145658, -123522.23770884426 6583368.372145658, -123522.23770884426 6584591.364598225, -124745.23016140609 6584591.364598225, -124745.23016140609 6583368.372145658))
BoundingBox(left=-177141.02370308578, bottom=6520436.993711313, right=1894.787500990933, top=6699916.905084926)
EPSG:3857
POLYGON ((-124745.23016140609 6582145.379693099, -123522.23770884426 6582145.379693099, -123522.23770884426 6583368.372145658, -124745.23016140609 6583368.372145658, -124745.23016140609 6582145.379693099))
BoundingBox(left=-177141.02370308578, bottom=6520436.993711313, right=1894.787500990933, top=6699916.905084926)
EPSG:3857
POLYGON ((-124745.23016140609 6580922.387240535, -123522.23770884426 6580922.387240535, -123522.23770884426 6582145.379693099, -124745.23016140609 6582145.379693099, -124745.23016140609 6580922.387240535))
BoundingBox(left=-177141.02370308578, bottom=6520436.993711313, right=1894.787500990933, top=6699916